# Первый этап - градиентный бустинг

In [1]:
import numpy as np
import pandas
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split, cross_val_score
import time
import datetime

In [2]:
train = pandas.read_csv('./data/features.csv', index_col='match_id')
test = pandas.read_csv('./data/features_test.csv', index_col='match_id')
X_test = test


In [3]:
train.drop(['duration','tower_status_radiant', 'tower_status_dire','barracks_status_radiant',
            'barracks_status_dire'], axis=1, inplace=True)
X_train = train.copy()
y_train = train['radiant_win']
X_train.drop(['radiant_win'], axis=1, inplace=True)

In [4]:
print "size:"
print len(X_train)
print X_train.count()

size:
97230
start_time                     97230
lobby_type                     97230
r1_hero                        97230
r1_level                       97230
r1_xp                          97230
r1_gold                        97230
r1_lh                          97230
r1_kills                       97230
r1_deaths                      97230
r1_items                       97230
r2_hero                        97230
r2_level                       97230
r2_xp                          97230
r2_gold                        97230
r2_lh                          97230
r2_kills                       97230
r2_deaths                      97230
r2_items                       97230
r3_hero                        97230
r3_level                       97230
r3_xp                          97230
r3_gold                        97230
r3_lh                          97230
r3_kills                       97230
r3_deaths                      97230
r3_items                       97230
r4_hero                   

In [5]:
X_train_missing = X_train.isnull().sum().sort_values(ascending=False)
print X_train_missing[X_train_missing > 0]

first_blood_player2            43987
radiant_flying_courier_time    27479
dire_flying_courier_time       26098
first_blood_time               19553
first_blood_team               19553
first_blood_player1            19553
dire_bottle_time               16143
radiant_bottle_time            15691
radiant_first_ward_time         1836
dire_first_ward_time            1826
radiant_courier_time             692
dire_courier_time                676
dtype: int64


#### "first blood" - убийство первого героя противника не так часто происходит в первые 5 минут игры. Предмет "flying_courier" cтановится доступным для покупки после 3 минут игры, поэтому это событие с меньшей вероятностью происходит в начале матча.

In [6]:
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

#### 'radiant_win'

In [7]:
kf = KFold(n_splits=5, shuffle=True)
kf.get_n_splits(y_train)
scores = list()
n = np.array([10, 20, 30, 100, 150])
for i in n:
    print i
    clf = GradientBoostingClassifier(n_estimators=i, random_state=1)
    start_time = datetime.datetime.now()
    score = cross_val_score(clf, X_train, y_train, cv=kf, scoring='roc_auc')
    print 'Time elapsed:', datetime.datetime.now() - start_time
    print score
    scores.append(np.mean(score))
print scores

10
Time elapsed: 0:00:18.225971
[ 0.66786601  0.66427695  0.65930366  0.6688307   0.66250008]
20
Time elapsed: 0:00:33.177820
[ 0.68077999  0.68397747  0.67945084  0.68162868  0.68659508]
30
Time elapsed: 0:00:48.548803
[ 0.68898572  0.68577347  0.68947223  0.69003901  0.69501449]
100
Time elapsed: 0:02:40.294670
[ 0.69782259  0.71119272  0.70618494  0.71024614  0.7049829 ]
150
Time elapsed: 0:03:57.885417
[ 0.70793896  0.71070899  0.71184107  0.70724822  0.71562737]
[0.66455548059959613, 0.68248641380267894, 0.68985698259423223, 0.70608586056516864, 0.71067292098511214]


#### ~49 секунд, качество, полученное для градиентного бустинга с 30 деревьями составляет 0.69

#### Имеет, но в разумных пределах. Чем больше количество деревьев, тем лучше качество мы получаем (например для 150 деревьев - 0.7107), но при бесконечном увеличении этого числа, мы стремимcя к предельному значению качества в ~0.72. Чтобы ускорить процесс можно проводить обучение на меньшем количестве данных или ограничить глубину деревьев с помощью параметра max_depth.

# Второй этап - логистическая регрессия

In [8]:
scores = list()

scaler = StandardScaler().fit(X_train)
n = np.array([0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000])

for i in n:
    print i
    clf = LogisticRegression(penalty='l2', C=i)
    start_time = datetime.datetime.now()
    score = cross_val_score(clf, scaler.transform(X_train), y_train, cv=kf, scoring='roc_auc')
    print 'Time elapsed:', datetime.datetime.now() - start_time
    print score
    scores.append(np.mean(score))
print scores

1e-05
Time elapsed: 0:00:02.274464
[ 0.69364753  0.69547581  0.70064405  0.69495789  0.69123879]
0.0001
Time elapsed: 0:00:03.774591
[ 0.70858221  0.71536879  0.71141844  0.71007877  0.71115932]
0.001
Time elapsed: 0:00:06.501475
[ 0.71343284  0.71699011  0.71492851  0.7138472   0.72203752]
0.01
Time elapsed: 0:00:08.674743
[ 0.71436529  0.71737277  0.71098858  0.72239199  0.71718288]
0.1
Time elapsed: 0:00:09.103536
[ 0.71635968  0.72049813  0.71920923  0.71427458  0.71191558]
1.0
Time elapsed: 0:00:09.116206
[ 0.71937401  0.71724919  0.71788862  0.71058863  0.71735001]
10.0
Time elapsed: 0:00:08.968399
[ 0.71717373  0.71137109  0.71874445  0.71426113  0.72007643]
100.0
Time elapsed: 0:00:09.058584
[ 0.70918845  0.71660828  0.71682669  0.72281244  0.71634228]
1000.0
Time elapsed: 0:00:09.180937
[ 0.71737504  0.71272958  0.71487704  0.71528179  0.72120711]
10000.0
Time elapsed: 0:00:09.217295
[ 0.71476217  0.71716021  0.72169752  0.71409446  0.71281683]
[0.69519281491042251, 0.71132150

####  Наилучшее качество получается при С=1 и равно 0.71649, что соотносится с качеством для градиентного бустинга при количестве деревьев > 100, и работает за время ~10 секунд.

In [9]:
X_train.drop(['lobby_type','r1_hero', 'r2_hero','r3_hero','r4_hero', 'r5_hero',
        'd1_hero', 'd2_hero','d3_hero','d4_hero', 'd5_hero'], axis=1, inplace=True)

y_new_train = train['radiant_win']
X_new_train = X_train

X_new_train = X_new_train.fillna(0)

kf = KFold(n_splits=5, shuffle=True)
kf.get_n_splits(y_new_train)

scaler = StandardScaler().fit(X_new_train)
n = np.array([0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000])
scores = list()
for i in n:
    print i
    clf = LogisticRegression(penalty='l2', C=i)
    score = cross_val_score(clf, scaler.transform(X_new_train), y_new_train, cv=kf, scoring='roc_auc')
    print score
    scores.append(np.mean(score))
print scores

1e-05
[ 0.69599126  0.6926869   0.69757039  0.68991055  0.69924683]
0.0001
[ 0.71350157  0.70869569  0.71617826  0.71038897  0.70746881]
0.001
[ 0.71374805  0.72117997  0.71684766  0.71316796  0.7166652 ]
0.01
[ 0.72086836  0.71739278  0.71308545  0.71609819  0.71448805]
0.1
[ 0.71082905  0.71636813  0.71884714  0.71663934  0.71847816]
1.0
[ 0.72125018  0.715705    0.71792564  0.71169184  0.71587666]
10.0
[ 0.71460818  0.71478439  0.71606445  0.7184986   0.7182985 ]
100.0
[ 0.71496183  0.71083225  0.7208133   0.71728106  0.7178246 ]
1000.0
[ 0.71855478  0.71926795  0.71437935  0.7195682   0.71081177]
10000.0
[ 0.71489044  0.71657124  0.71388462  0.72325736  0.71321947]
[0.6950811857700907, 0.71124665846966484, 0.71632176689898608, 0.71638656743403895, 0.71623236340517105, 0.71648986515610402, 0.71645082446662434, 0.71634260775272107, 0.71651641105003061, 0.71636462831282255]


#### Качество модели практически не изменилось и составило  0.71651 при C=1000, значит при обучении метод счёл их малозначимыми.

In [10]:
hero_table = train[['r1_hero', 'r2_hero','r3_hero','r4_hero', 'r5_hero',
        'd1_hero', 'd2_hero','d3_hero','d4_hero', 'd5_hero']].values

print hero_table
print len(np.unique(hero_table))

[[ 11  67  29 ...,  21  37  84]
 [ 42  49  67 ...,  79   7  12]
 [ 33  98  20 ...,  86  29  80]
 ..., 
 [ 98  11 112 ...,  55  59  31]
 [100  72  79 ...,  50  28 106]
 [ 50  19  84 ...,  87 112  97]]
108


#### Всего 108 различных героев используется в предоставленных данных о матчах из возможных 112.

In [11]:
heroes = pandas.read_csv('./data/dictionaries/heroes.csv')
N = len(heroes)

X_pick = np.zeros((train.shape[0], N))

for i, match_id in enumerate(train.index):
    for p in xrange(5):
        X_pick[i, train.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, train.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
X_hero = pandas.DataFrame(X_pick, index=X_new_train.index)
X_new_train = pandas.concat([X_new_train, X_hero], axis=1)
print X_new_train

          start_time  r1_level  r1_xp  r1_gold  r1_lh  r1_kills  r1_deaths  \
match_id                                                                     
0         1430198770         5   2098     1489     20         0          0   
1         1430220345         4   1188     1033      9         0          1   
2         1430227081         4   1319     1270     22         0          0   
3         1430263531         4   1779     1056     14         0          0   
4         1430282290         4   1431     1090      8         1          0   
5         1430284186         5   1961     1461     19         0          1   
8         1430293701         3    967     1136      7         1          0   
9         1430299335         5   2117     1252     16         0          0   
11        1430308974         5   1527      906     10         0          1   
12        1430316105         5   1651     1060     14         0          1   
14        1430325079         5   1988     1804     21         1 

In [12]:
kf = KFold(n_splits=5, shuffle=True)
kf.get_n_splits(y_new_train)
scores = list()

scaler = StandardScaler()
n = np.array([0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000])

for i in n:
    print i
    clf = LogisticRegression(penalty='l2', C=i)
    score = cross_val_score(clf, scaler.fit_transform(X_new_train), y_new_train, cv=kf, scoring='roc_auc')
    print score
    scores.append(np.mean(score))
print scores

1e-05
[ 0.71843234  0.7156102   0.71385139  0.71088601  0.71551902]
0.0001
[ 0.74075469  0.74737403  0.7383318   0.74287532  0.74400491]
0.001
[ 0.74635001  0.75090198  0.7576647   0.75262535  0.7503239 ]
0.01
[ 0.75375239  0.75227313  0.74873902  0.75181542  0.7531222 ]
0.1
[ 0.75049137  0.75708429  0.74885475  0.75190454  0.75019258]
1.0
[ 0.75459088  0.74939511  0.7513796   0.75059092  0.75327184]
10.0
[ 0.74950101  0.75653499  0.75333634  0.75347756  0.74753082]
100.0
[ 0.75093868  0.75384154  0.75171322  0.75102651  0.7518409 ]
1000.0
[ 0.75528206  0.75518696  0.74773371  0.74972502  0.75086866]
10000.0
[ 0.75085045  0.75337732  0.74734378  0.75772736  0.74959236]
[0.71485979178647585, 0.74266815158640154, 0.75157318745114465, 0.75194043068518268, 0.75170550759008958, 0.75184566992436364, 0.75207614279178325, 0.75187216863084216, 0.75175928292962313, 0.75177825314387792]


#### При добавлении "мешка слов" по героям качество модели улучшилось и равно 0.7521 при C = 10. При one-hot-кодировании, линейная модель получает пригодные для предсказания данные в виде вещественных векторов, что упрощает построение разделяющей плоскости.

In [13]:
test = pandas.read_csv('./data/features_test.csv', index_col='match_id')
test = test.fillna(0)
X_test = test.copy()

X_test.drop(['lobby_type','r1_hero', 'r2_hero','r3_hero','r4_hero', 'r5_hero',
        'd1_hero', 'd2_hero','d3_hero','d4_hero', 'd5_hero'], axis=1, inplace=True)

X_pick = np.zeros((test.shape[0], N))

for i, match_id in enumerate(test.index):
    for p in xrange(5):
        X_pick[i, test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
X_hero = pandas.DataFrame(X_pick, index=X_test.index)
X_new_test = pandas.concat([X_test, X_hero], axis=1)
print X_new_test


clf = LogisticRegression(penalty='l2', C=10)
clf.fit(scaler.fit_transform(X_new_train), y_new_train)
pred = clf.predict_proba(scaler.fit_transform(X_new_test))[:, 1]

print pred.min()
print pred.max()
kaggle = pandas.DataFrame({'radiant_win': pred}, index=X_test.index)
print kaggle
kaggle.to_csv("dota_win_pred.csv")

          start_time  r1_level  r1_xp  r1_gold  r1_lh  r1_kills  r1_deaths  \
match_id                                                                     
6         1430287923         4   1103     1089      8         0          1   
7         1430293357         2    556      570      1         0          0   
10        1430301774         2    751      808      1         0          0   
13        1430323933         3    708      903      1         1          1   
16        1430331112         4   1259      661      4         0          0   
18        1430334264         5   2067     1549     15         0          0   
19        1430334995         4   1862     1487     24         0          1   
24        1430349466         5   1991     1983     19         2          0   
33        1430366484         2    517      500      0         0          1   
37        1430375099         3   1060      691      5         0          0   
41        1430378434         4   1760     1162     16         0 

0.00868998577634
0.996487267284
          radiant_win
match_id             
6            0.824995
7            0.758552
10           0.186991
13           0.859399
16           0.237548
18           0.379512
19           0.530337
24           0.564540
33           0.212069
37           0.674474
41           0.154922
42           0.324287
55           0.233739
60           0.675504
62           0.552531
63           0.613138
64           0.087765
71           0.556069
72           0.315564
83           0.505678
85           0.783118
89           0.932450
92           0.757473
100          0.960659
102          0.885839
108          0.534843
111          0.774635
126          0.186858
130          0.052473
140          0.791328
...               ...
114165       0.815251
114168       0.667554
114171       0.462899
114183       0.145236
114188       0.633485
114192       0.843475
114195       0.519012
114202       0.479319
114209       0.385393
114211       0.809121
114217       0.754378


#### Значения 0.0087 и 0.9965 соответственно.